In [4]:
import json
from authorization import SpotifyAuth

In [5]:
with open('client-secrets-rnr.json','r+') as secrets_file:
    secrets = json.load(secrets_file)

client_id = secrets["client_id"]
client_secret = secrets["client_secret"]
redirect_url = secrets["redirect_url"]

In [6]:
spotify_authorisation = SpotifyAuth(client_id=client_id, client_secret=client_secret, redirect_url=redirect_url)
spotify_authorisation.set_access_token_from_file('access_token.json')

# Get User Details

In [7]:
import spotify_functions as spotify

In [8]:
spotify.get_user_details(spotify_authorisation.get_access_token())

{'display_name': 'sdspot2034',
 'external_urls': {'spotify': 'https://open.spotify.com/user/sdspot2034'},
 'href': 'https://api.spotify.com/v1/users/sdspot2034',
 'id': 'sdspot2034',
 'images': [{'url': 'https://i.scdn.co/image/ab67757000003b8280153af8b79b3144b24f1291',
   'height': 64,
   'width': 64},
  {'url': 'https://i.scdn.co/image/ab6775700000ee8580153af8b79b3144b24f1291',
   'height': 300,
   'width': 300}],
 'type': 'user',
 'uri': 'spotify:user:sdspot2034',
 'followers': {'href': None, 'total': 9},
 'country': 'IN',
 'product': 'premium',
 'explicit_content': {'filter_enabled': False, 'filter_locked': False},
 'email': 'shreyandas2034@gmail.com'}

In [9]:
import datetime
yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
# print(yesterday.timestamp()*1e3)
cdc_time = int(yesterday.timestamp()*1e3)

# Get All Songs Played in the last one day

In [10]:
token = spotify_authorisation.get_access_token()
history = spotify.get_listening_history(token, cdc_time)['items']

## EDA over recent history

### Number of songs played

In [11]:
len(history)

20

### Fetch details of a song

#### See available data for a song

In [12]:
history[0].keys()

dict_keys(['track', 'played_at', 'context'])

In [13]:
list(history[0]['track'].keys())

['album',
 'artists',
 'disc_number',
 'duration_ms',
 'explicit',
 'external_ids',
 'external_urls',
 'href',
 'id',
 'is_local',
 'name',
 'popularity',
 'preview_url',
 'track_number',
 'type',
 'uri']

In [14]:
history[0]['played_at']

'2024-08-02T14:53:41.386Z'

In [15]:
print(history[0]['context'])

None


# Transform

In [16]:
import pandas as pd
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData
from sqlalchemy.dialects.mysql import insert

## Dimensions

### DIM_ARTIST

In [17]:
# stacked list-comprehension
artist_list = [artist for artists in [play['track']['artists'] for play in history] for artist in artists]

In [18]:
df_artists = pd.DataFrame(artist_list)
df_artists = df_artists.drop(columns=['href','uri','external_urls'])
df_artists = df_artists.rename(columns={'id':'spotify_id','name':'artist_name','type':'artist_type'})
df_artists.drop_duplicates(inplace=True)
df_artists

,spotify_id,artist_name,artist_type
0,25u4wHJWxCA9vO0CzxAbK7,Lukas Graham,artist
1,4gzpq5DPGxSnKTe4SA8HAU,Coldplay,artist
3,2wY79sveU1sp5g7SokKOiI,Sam Smith,artist
4,33NVpKoXjItPwUJTMZIOiY,David Kushner,artist
7,5pKCCKE2ajJHZ9KAiaK11H,Rihanna,artist
8,1buzCmyYZE4kcdLRudsb8V,Mikky Ekko,artist
9,7t0rwkOPGlDPEhaOcVtOt9,The Cranberries,artist
10,72OaDtakiy6yFqkt4TsiFt,Cher,artist
11,74rnJF4h8ScpbyEhLholBg,Archer Marsh,artist
12,0TnOYISbd1XYRBk9myaseg,Pitbull,artist


#### Load (Type 1 SCD)

In [19]:
with open('SQL_CREDENTIALS.json', 'r') as file:
    cred = json.loads(file.read())
    
user = cred['user']
pswd = cred['pass']
host = cred['host']

target_engine = create_engine(f'mysql+mysqlconnector://{user}:{pswd}@{host}:3306/spotify_db')


In [27]:
metadata = MetaData()

# Define the dimension table schema
dim_table = Table('DIM_ARTIST', metadata,
    Column('artist_wid', Integer, primary_key=True, autoincrement=True),
    Column('spotify_id', String(255), nullable=False, unique=True),
    Column('artist_name', String(255)),
    Column('artist_type', String(255))
)

In [29]:
list_of_records = df_artists.to_dict(orient='records')

with target_engine.connect() as connection:
    # Create the insert statement with ON DUPLICATE KEY UPDATE
    insert_stmt = insert(dim_table).values(list_of_records)
    upsert_stmt = insert_stmt.on_duplicate_key_update(
        artist_name=insert_stmt.inserted.artist_name,
        artist_type=insert_stmt.inserted.artist_type
    )

    # Execute the upsert statement
    connection.execute(upsert_stmt)

In [75]:
with open('SQL_CREDENTIALS.json', 'r') as file:
    cred = json.loads(file.read())
    
user = cred['user']
pswd = cred['pass']
host = cred['host']

target_engine = create_engine(f'mysql+mysqlconnector://{user}:{pswd}@{host}:3306/spotify_db')

with target_engine.connect() as connection:
    dim_artist = pd.read_sql_table("DIM_ARTIST", con=connection)
    
df_combined = df_artists.join(dim_artist, on=["spotify_id"], how="left", rsuffix='_dim')
df_inserts = df_combined[df_combined['artist_wid'].isnull()][['spotify_id','artist_name','artist_type']]
df_updates = df_combined.dropna(subset=['artist_wid'])

with target_engine.connect() as connection:
    df_inserts.to_sql(name='DIM_ARTIST', con=connection, if_exists='append', index=False)

### DIM_SONG

In [17]:
history[0]['track'].keys()

dict_keys(['album', 'artists', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [18]:
history[0]['track']

{'album': {'album_type': 'single',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5pKCCKE2ajJHZ9KAiaK11H'},
    'href': 'https://api.spotify.com/v1/artists/5pKCCKE2ajJHZ9KAiaK11H',
    'id': '5pKCCKE2ajJHZ9KAiaK11H',
    'name': 'Rihanna',
    'type': 'artist',
    'uri': 'spotify:artist:5pKCCKE2ajJHZ9KAiaK11H'},
   {'external_urls': {'spotify': 'https://open.spotify.com/artist/5K4W6rqBFWDnAN6FQUkS6x'},
    'href': 'https://api.spotify.com/v1/artists/5K4W6rqBFWDnAN6FQUkS6x',
    'id': '5K4W6rqBFWDnAN6FQUkS6x',
    'name': 'Kanye West',
    'type': 'artist',
    'uri': 'spotify:artist:5K4W6rqBFWDnAN6FQUkS6x'},
   {'external_urls': {'spotify': 'https://open.spotify.com/artist/4STHEaNw4mPZ2tzheohgXB'},
    'href': 'https://api.spotify.com/v1/artists/4STHEaNw4mPZ2tzheohgXB',
    'id': '4STHEaNw4mPZ2tzheohgXB',
    'name': 'Paul McCartney',
    'type': 'artist',
    'uri': 'spotify:artist:4STHEaNw4mPZ2tzheohgXB'}],
  'external_urls': {'spotify': 'https://open.s